<a href="https://colab.research.google.com/github/jianzhiw/NLP/blob/master/Simple_NLTK_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Pre- Processing with NLTK #



*  Converting the entire text into uppercase or lowercase, so that the algorithm does not treat the same words in different cases as different
* Tokenization: Tokenization is just the term used to describe the process of converting the normal text strings into a list of tokens i.e words that we actually want. Sentence tokenizer can be used to find the list of sentences and Word tokenizer can be used to find the list of words in strings.
* Removing Noise i.e everything that isn’t in a standard number or letter.
* Removing Stop words. Sometimes, some extremely common words which would appear to be of little value in helping select documents matching a user need are excluded from the vocabulary entirely. These words are called stop words
* Stemming: Stemming is the process of reducing inflected (or sometimes derived) words to their stem, base or root form — generally a written word form. Example if we were to stem the following words: “Stems”, “Stemming”, “Stemmed”, “and Stemtization”, the result would be a single word “stem”.
* Lemmatization: A slight variant of stemming is lemmatization. The major difference between these is, that, stemming can often create non-existent words, whereas lemmas are actual words. So, your root stem, meaning the word you end up with, is not something you can just look up in a dictionary, but you can look up a lemma. Examples of Lemmatization are that “run” is a base form for words like “running” or “ran” or that the word “better” and “good” are in the same lemma so they are considered the same.

# Bag of Words #
After the initial preprocessing phase, we need to transform text into a meaningful vector (or array) of numbers. The bag-of-words is a representation of text that describes the occurrence of words within a document. It involves two things:
* A vocabulary of known words.
* A measure of the presence of known words.
<br></br>

Why is it is called a “bag” of words? That is because any information about the order or structure of words in the document is discarded and the model is only concerned with whether the known words occur in the document, not where they occur in the document.
<br></br>

The intuition behind the Bag of Words is that documents are similar if they have similar content. Also, we can learn something about the meaning of the document from its content alone.
<br></br>

For example, if our dictionary contains the words {Learning, is, the, not, great}, and we want to vectorize the text “Learning is great”, we would have the following vector: (1, 1, 0, 0, 1).

# TF-IDF Approach #
A problem with the Bag of Words approach is that highly frequent words start to dominate in the document (e.g. larger score), but may not contain as much “informational content”. Also, it will give more weight to longer documents than shorter documents.
<br></br>

One approach is to rescale the frequency of words by how often they appear in all documents so that the scores for frequent words like “the” that are also frequent across all documents are penalized. This approach to scoring is called Term Frequency-Inverse Document Frequency, or TF-IDF for short, where:
<br></br>
* Term Frequency: is a scoring of the frequency of the word in the current document.
<br></br>
> TF = (Number of times term t appears in a document)/(Number of terms in the document)
* Inverse Document Frequency: is a scoring of how rare the word is across documents.
> IDF = 1+log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
<br></br>

Tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus

## Example ##
Consider a document containing 100 words wherein the word ‘phone’ appears 5 times.
<br></br>
The term frequency (i.e., tf) for phone is then (5 / 100) = 0.05. Now, assume we have 10 million documents and the word phone appears in one thousand of these. Then, the inverse document frequency (i.e., IDF) is calculated as 
<br></br>
$ log(10,000,000 / 1,000) = 4. $
<br></br>
Thus, the Tf-IDF weight is the product of these quantities: 0.05 * 4 = 0.20.
Tf-IDF can be implemented in scikit learn as:
> from sklearn.feature_extraction.text import TfidfVectorizer

# Cosine Similarity #
TF-IDF is a transformation applied to texts to get two real-valued vectors in vector space. We can then obtain the [Cosine similarity](https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/?source=post_page---------------------------) of any pair of vectors by taking their dot product and dividing that by the product of their norms. 
That yields the cosine of the angle between the vectors. Cosine similarity is a measure of similarity between two non-zero vectors. Using this formula we can find out the similarity between any two documents d1 and d2.
> Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||

where d1,d2 are two non zero vectors.

In [0]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import warnings
warnings.filterwarnings("ignore")


# Retrieving the dataset #

We are going to use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) API to retrieve the text from Wikipedia.

In [0]:
from bs4 import BeautifulSoup
import requests
import re

respond = requests.get("https://en.wikipedia.org/wiki/Chatbot")

soup = BeautifulSoup(respond.text)
l = soup.find_all('p')
text = ""
#f = open("chatbot.txt", mode="a")
for item in l:
  #f.write(item.text)
  text += item.text

text = re.sub(r'\[\d*\]', "", text)
f = open("chatbot.txt", mode="w")
f.write(text)

15173

# Reading in the data #
We will read in the corpus.txt file and convert the entire corpus into a list of sentences and a list of words for further pre-processing.

In [0]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
sent_tokens[:2]

['a chatbot is a computer program which conducts a conversation via auditory or textual methods.',
 'such programs are often designed to convincingly simulate how a human would behave as a conversational partner, although as of 2019, they are far short of being able to pass the turing test.']

In [0]:
word_tokens[:2]

['a', 'chatbot']

# Pre-processing the raw text #
We shall now define a function called LemTokens which will take as input the tokens and return normalized tokens.

In [0]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Keyword matching #
Next, we shall define a function for a greeting by the bot i.e if a user’s input is a greeting, the bot shall return a greeting response.ELIZA uses a simple keyword matching for greetings. We will utilize the same concept here.

In [0]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Generating Response #
To generate a response from our bot for input questions, the concept of document similarity will be used. So we begin by importing necessary modules.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

We define a function response which searches the user’s utterance for one or more known keywords and returns one of several possible responses. If it doesn’t find the input matching any of the keywords, it returns a response:” I am sorry! I don’t understand you”

In [0]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Finally, we will feed the lines that we want our bot to say while starting and ending a conversation depending upon user’s input.

In [0]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hi
ROBO: hi there
what is chatterbot
ROBO: the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describe these conversational programs.
who launch sobot
ROBO: the france's third largest bank by total assets  société générale launched their chatbot called sobot in march 2018. while the 80% of users of the sobot express their satisfaction after having tested it, il will never remplace the expertise provided by a human advisor according to sg deputy director bertrand cozzarolo.
bye
ROBO: Bye! take care..


# Reference #
[Source](https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e)

Prepared By: [Wong Jian Zhi](https://github.com/jianzhiw)